In [3]:
from PIL import Image
from PIL import Image, ImageFont, ImageDraw 
import random
import cv2
import numpy as np
from numpy import asarray

In [4]:
def set_coordinates(max_x, max_y, coordinates = None):
    if coordinates is None:
        coordinates = (random.randint(0, max_x), random.randint(0, max_y))
    return coordinates

def set_color(color = None):
    if color is None:
        color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    return color

def set_size(size = None):
    if size is None:
        size = random.randint(0, 500)
    return size

def set_text(text = None):
    if text is None:
        # TODO сделать вывод рандомной фразы из заготвленного списка
        text = 'Test'
    return text

class Editor:
    def __init__(self, path_to_image):
        self.__image = Image.open(path_to_image)
        
    def get_image(self):
        return self.__image
    
    def set_image(self, path_to_image):
        self.__image = Image.open(path_to_image)
        
    def rotate(self, angle = None):
        if angle is None:
            angle = random.randint(0, 360)
        self.__image = self.__image.rotate(angle)
        
    def add_text(self, coordinates = None, text = None, color = None, font = None, font_size = None):
        #TODO разобраться какие брать максимальные границы, пока 500х500
        height, width = self.__image.size
        coordinates = set_coordinates(500, 500, coordinates)
        text = set_text(text)
        color = set_color(color)
            
        if font is None:
            #TODO выбрать рандомный шрифт
            if font_size is None:
                font_size = random.randint(0, 400)
            font = ImageFont.truetype('/home/marat/fonts/PlayfairDisplay-Black.ttf', font_size)
        
        try:
            ImageDraw.Draw(self.__image).text(coordinates, text, color, font)
        except TypeError:
            ImageDraw.Draw(self.__image).text(coordinates, text, random.randint(0, 255), font)
        
    def add_figures(self, figures):
        self.__image = FiguresEditor.add_figures(self.__image, figures)
        
    def add_filter(self, image_filter):
        self.__image = FilterEditor.add_filter(self.__image, image_filter)

In [95]:
class FiguresEditor:
    figures_dict = {
        'circle': 'add_cirles',
        'rombus': 'add_rombuses',
        'heart': 'add_hearts',
        'star': 'add_stars'
    }
    
    @staticmethod
    def add_figures(image, user_figure_dict):
        for user_figure_name, user_figure_options_dict in user_figure_dict.items():
            image = getattr(FiguresEditor, FiguresEditor.figures_dict[user_figure_name])(image, user_figure_options_dict)
        
        return image
            
    def add_cirles(image, user_figure_options_dict):
        data = asarray(image)
        for user_figure_option in user_figure_options_dict:
            coordinates = set_coordinates(500, 500, user_figure_option['coordinates'])
            size = set_size(user_figure_option['size'])
            color = set_color(user_figure_option['color'])
            
            cv2.circle(data, coordinates, size, color, 10)
        image = Image.fromarray(data)
        
        return image
    
    def add_stars(image, user_figure_options_dict):
        for user_figure_option in user_figure_options_dict:
            star = Image.open('/home/marat/projects/MagicPhotos/figures/star.png', 'r').convert("RGBA")
            data = np.array(star)
            
            coordinates = set_coordinates(500, 500, user_figure_option['coordinates'])
            size = set_size(user_figure_option['size'])
            color = set_color(user_figure_option['color'])
        
            r1, g1, b1 = 0, 0, 0 # Original value
            r2, g2, b2 = color # Value that we want to replace it with

            red, green, blue = data[:,:,0], data[:,:,1], data[:,:,2]
            mask = (red == r1) & (green == g1) & (blue == b1)
            data[:,:,:3][mask] = [r2, g2, b2]

            star = Image.fromarray(data)

            star.thumbnail((size, size), Image.ANTIALIAS)
            star = star.rotate(random.randint(0, 360))

            image.paste(star, coordinates, mask = star)
        
        return image

In [13]:
class FilterEditor:
    filter_dict = {
        'black_and_white': 'make_black_and_white',
        'sepia': 'make_sepia',
        'bright': 'make_bright'
    }
    
    @staticmethod
    def add_filter(image, user_filter):
        #TODO Проверка на тот случай, если передавать словарь с настройками для цвета, но это в будущем
        #if isinstance(user_filter, dict):
            #return getattr(FilterEditor, FilterEditor.filter_dict[user_filter.keys()[0]])(image, user_filter.values()[0])
        return getattr(FilterEditor, FilterEditor.filter_dict[user_filter])(image)
        
    def make_black_and_white(image):
        result = Image.new('RGB', image.size)
        for x in range(image.size[0]):
            for y in range(image.size[1]):
                r, g, b = image.getpixel((x, y))
                gray = int(r * 0.2126 + g * 0.7152 + b * 0.0722)
                result.putpixel((x, y), (gray, gray, gray))
        return result
    
    def make_sepia(image):
        result = Image.new('RGB', image.size)
        for x in range(image.size[0]):
            for y in range(image.size[1]):
                r, g, b = image.getpixel((x, y))
                red = int(r * 0.393 + g * 0.769 + b * 0.189)
                green = int(r * 0.349 + g * 0.686 + b * 0.168)
                blue = int(r * 0.272 + g * 0.534 + b * 0.131)
                result.putpixel((x, y), (red, green, blue))
        return result
    
    def make_bright(image):
        brightness = 2
        result = Image.new('RGB', image.size)
        for x in range(image.size[0]):
            for y in range(image.size[1]):
                r, g, b = image.getpixel((x, y))

                red = int(r * brightness)
                red = min(255, max(0, red))

                green = int(g * brightness)
                green = min(255, max(0, green))

                blue = int(b * brightness)
                blue = min(255, max(0, blue))

                result.putpixel((x, y), (red, green, blue))
        return result

In [ ]:
#TODO Подставляется путь к изображению
editor = Editor('/home/marat/Рабочий стол/slid.jpeg')

editor.rotate()
editor.add_text()
editor.add_filter('black_and_white')

figures_dict = {
    'circle': [
        {
            'color': None,
            'size': None,
            'coordinates': None
        },
        {
            'color': None,
            'size': None,
            'coordinates': None
        }
    ], 
    'star': [
        {
            'color': (150,150,200),
            'size': None,
            'coordinates': (10, 10)
        },
        {
            'color': None,
            'size': None,
            'coordinates': None
        }
    ]
}
editor.add_figures(figures_dict)

editor.get_image().show()